# AceleraDev DataScience

## Setup

kaggle: House Prices Advanced regression techniques

## ML Canvas

### Qual é a proposta de valor?
O que é esperado do modelo, para que ele servirá, para quem servirá.

Ex: Prever o preço do imóvel com base nas variáveis x,y,z

### Para que tipo de decisões o modelo será usado?

Será necessário calcular de quanto em quanto tempo?

### Quais são minhas fontes de dados?

- Que tipos de fontes?
- São internas? Externas?
- Nacional, Internacional?
- Como os dados serão coletados?
- Qual a complexidade de obter os dados?

As fontes não precisar ser fixas, é possível incluir remover fontes no decorrer do desenvolvimento, mas é necessário fazer uma discussão inicial

### Discussão de modelos

- Levanta mento de modelos, métodos, técnicas.

### Avaliações prévias

Métodos e métricas para avaliar o sistema antes dele ser empregado.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
df = pd.read_csv("train.csv")

## O cientista deve enter absolutamente tudo sobre os dados
O que significa cada coluna e os valores que elas trazem.

In [ ]:
df.columns

In [ ]:
df.describe()

In [ ]:
df.dtypes

In [ ]:
cons = pd.DataFrame({'colunas':df.columns, 'tipo':df.dtypes,'missing':df.isna().sum(),'size':df.shape[0], 'unicos':df.nunique()})
cons['percentual'] = round(cons['missing']/cons['size'], 2)
cons

In [ ]:
cons.percentual.plot.hist(bins=5)

In [ ]:
print('Quantidade de colunas com ATÈ 20% dos dados faltantes: ', cons[cons.percentual < 0.2].shape[0])
print('Quantidade de colunas com 0% dos dados faltantes: ', cons[cons.percentual == 0].shape[0])

In [ ]:
cons[cons.percentual == 0].tipo.value_counts()

In [ ]:
cons['completa'] = cons['percentual'].apply(lambda x: 'completa' if x == 0  else 'faltante')
cons

In [ ]:
df = df[cons[cons.completa == 'completa'].colunas]

In [ ]:
df

In [ ]:
colunas_numericas = list(cons[(cons['tipo'] != 'object') & (cons['completa'] == 'completa')]['colunas'])

In [ ]:
for coluna in colunas_numericas:
    print(coluna)
    df[coluna].plot.hist(bins=50, log=True)
    plt.show()

In [ ]:
plt.figure(figsize=(20,20))
sns.heatmap(df[colunas_numericas].corr().round(2), annot=True)

In [ ]:
correlacionadas=['GarageArea', 'GarageCars', 'GrLivArea', 'OverallQual']

In [ ]:
colunas_numericas.remove('Id')

In [ ]:
df = df[colunas_numericas]

In [ ]:
y_train = df['SalePrice']

In [ ]:
x_train = df.drop(columns='SalePrice')

In [ ]:
from yellowbrick.features import Rank1D
visualizer = Rank1D(algorithm='shapiro')
visualizer.fit(x_train, y_train)
visualizer.transform(x_train)
visualizer.show()

In [ ]:
from yellowbrick.features import PCA

visualizer = PCA(scale=True, proj_features=True, projection=2)
visualizer.fit_transform(x_train[correlacionadas], y_train)
visualizer.show()

In [ ]:
from yellowbrick.target import FeatureCorrelation

features = list(x_train.columns)

visualizer = FeatureCorrelation(labels=features)
visualizer.fit(x_train, y_train)
visualizer.show()

# Treinando o modelo

In [ ]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression()

reg.fit(x_train, y_train)

In [ ]:
colunas_treinamento = x_train.columns

X_test = pd.read_csv('test.csv')
y_test = pd.read_csv('sample_submission.csv')

In [ ]:
y_test = y_test['SalePrice']

In [ ]:
X_test = X_test[colunas_treinamento].fillna(df[colunas_treinamento].mean())

In [ ]:
y_pred = reg.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
erro_normal = mean_squared_error(y_pred = y_pred, y_true=y_test)

In [ ]:
erro_normal

In [ ]:
sns.scatterplot(y_pred, y_test)

## Aplicando o Feature Selection

In [ ]:
from sklearn.feature_selection import RFE

In [ ]:
rfe= RFE(reg)

In [ ]:
rfe.fit(x_train, y_train)

In [ ]:
pd.DataFrame({'coluna': x_train.columns, 'bool':rfe.get_support(), 'coeficientes':pd.Series(reg.coef_)})

In [ ]:
X_train_importance = rfe.transform(x_train)
X_test_importance = rfe.transform(X_test)

In [ ]:
reg.fit(X_train_importance, y_train)

In [ ]:
y_pred_imp = reg.predict(X_test_importance)

In [ ]:
erro_imp = mean_squared_error(y_pred=y_pred_imp, y_true=y_test)

In [ ]:
erro_imp

In [ ]:
from yellowbrick.model_selection import RFECV

In [ ]:
visualizer = RFECV(reg)
visualizer.fit(x_train, y_train)
visualizer.show()

## Aplicando PCA

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(0.95)

In [ ]:
pca.fit(x_train)

In [ ]:
pca.explained_variance_ratio_

In [ ]:
x_train_pca = pca.transform(x_train)
X_test_pca = pca.transform(X_test)

In [ ]:
reg = LinearRegression()
reg.fit(x_train_pca, y_train)

In [ ]:
y_pred_pca = reg.predict(X_test_pca)

In [ ]:
erro_pca = mean_squared_error(y_pred=y_pred_pca, y_true=y_test)

In [ ]:
erro_pca

In [ ]:
pd.DataFrame({'erro':[erro_normal, erro_imp, erro_pca]}).plot(kind='bar', log=True)